# MDLE: Assignment 1
## Preprocessing
Students were provided with the dataset `conditions.csv.gz` which lists conditions for a large set of patients. The file contains the following fields, with multiple non-consecutive entries for each patient:

START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
- PATIENT is the patient identifier
- CODE is a condition identifier
- DESCRIPTION is the name of the condition

As each record contains an encountered condition, the purpose of this notebook is to preprocess the data by aggregating all the conditions found in each patient into a single record and then save the resulting dataset to disk as a compressed parquet file.

In [1]:
INPUT_FILE_PATH = "data/conditions.csv.gz"
OUTPUT_FILE_PATH = "data/conditions.parquet"

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("A-Priori").getOrCreate()

In [3]:
data = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("compression", "gzip") \
    .load(INPUT_FILE_PATH)


In [4]:
data.show()

+----------+----------+--------------------+--------------------+---------+--------------------+
|     START|      STOP|             PATIENT|           ENCOUNTER|     CODE|         DESCRIPTION|
+----------+----------+--------------------+--------------------+---------+--------------------+
|2017-01-14|2017-03-30|09e4e8cb-29c2-4ef...|88e540ab-a7d7-47d...| 65363002|        Otitis media|
|2012-09-15|2012-09-16|b0a03e8c-8d0f-424...|e89414dc-d0c6-478...|241929008|Acute allergic re...|
|2018-06-17|2018-06-24|09e4e8cb-29c2-4ef...|c14325b0-f7ec-431...|444814009|Viral sinusitis (...|
|2019-04-19|2019-09-26|09e4e8cb-29c2-4ef...|71af18ee-3157-408...| 65363002|        Otitis media|
|2019-04-27|2019-05-18|09e4e8cb-29c2-4ef...|411d4eae-72d1-478...|444814009|Viral sinusitis (...|
|2019-06-03|2019-08-02|09e4e8cb-29c2-4ef...|667a94d9-6aa1-4b6...| 33737001|     Fracture of rib|
|2014-11-09|2014-11-30|b0a03e8c-8d0f-424...|53431016-43c6-46b...|444814009|Viral sinusitis (...|
|2015-01-04|2015-01-18|b0a03e8

#### Group entries by patient and collect all codes as a list for each patient

In [5]:
from pyspark.sql.functions import collect_set

conditions = data.select("PATIENT", "CODE") \
    .groupBy("PATIENT") \
    .agg(collect_set("CODE").alias("CONDITIONS"))

conditions.show()

+--------------------+--------------------+
|             PATIENT|          CONDITIONS|
+--------------------+--------------------+
|000134b3-b39d-47c...|[271737000, 20183...|
|000217d4-7ed6-4b9...|[70704007, 65363002]|
|0002b3ea-a505-416...|[162864005, 10509...|
|00041ad6-1cdd-401...|[271737000, 10509...|
|00047a67-6ec3-4e7...|[703151001, 36971...|
|0004846e-17b1-41e...|[84757009, 370247...|
|00053fa0-3f76-4b9...|[87433001, 429007...|
|000555dc-8e3d-4dc...|[72892002, 400550...|
|000562e7-d1d7-4e7...|[162864005, 59621...|
|0005c4be-6202-421...|[162864005, 26929...|
|0006671b-6c1b-482...|[446096008, 72892...|
|000668eb-910f-486...|[162864005, 42825...|
|00070946-d03b-41f...|[713197008, 27173...|
|00073faf-a883-460...|[241929008, 44609...|
|00095476-44f2-4b8...|[271737000, 10509...|
|000a89f1-ab45-4a7...|[59621000, 161140...|
|000c1354-8f8e-4e9...|[195662009, 19212...|
|000cdee6-e734-4e9...|[62564004, 429007...|
|000d229a-ce75-416...|[72892002, 105090...|
|000dbb53-a0fe-43a...|[65966004,

#### Save data to parquet
While CSV does not support nested data structures, Parquet allows us to store columns with lists or sets e.g. `set<string>`.

In [ ]:
conditions.write \
    .mode("overwrite") \
    .format("parquet") \
    .option("compression", "gzip") \
    .save(OUTPUT_FILE_PATH)

#### Delete the Spark Session

In [ ]:
spark.stop()